In [2]:
g = 2
n = 2
indexes = [str(i) for i in range(1, n+1)]

We first create the ring of function on the strongly non-special locus. For now, let's stick to the genus two case, so we don't need the $c_{ij}^k$.

In [3]:
def one_index_names(name):
	return [name +"_"+ i  for i in indexes]
	
def two_index_names(name):
	return [name +"_"+ i + j for i in indexes for j in indexes if i != j]

def two_index_names_repeat(name):
    names = []
    for i in indexes:
        for j in indexes:
            if i != j:
                names.append(name + "_" + i + j)
            else:
                for k in indexes:
                    if k != i:
                        T = [i,k]
                        T.sort()
                        names.append(name + "_" + i + i + "_" + "".join(T))
    return names
    #return [name +"_"+ i + j for i in indexes for j in indexes]

def three_index_names(name):
	return [name +"_"+ i + j + k for i in indexes for j in indexes for k in indexes if i != j and i != k and j !=k]

def jjpS_names(name):
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for jj in Subsets(indexes,2):
            jjs = list(jj)
            jjs.sort()
            j = jjs[0]
            jp = jjs[1]
            
            if j not in S and jp not in S:
                names.append(name + "_" + j + jp + "_" + "".join(Ss))
                names.append(name + "_" + jp + j + "_" + "".join(Ss))

    return names

def ijS_names(name):
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for i in Ss:
            for j in indexes:
                if j not in Ss:
                    names.append(name + "_" + i + j + "_" + "".join(Ss))

    return names

def iS_names(name):
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for i in Ss:
            names.append(name + "_" + i + "_" + "".join(Ss))
    return names

def inotinS_names(name):
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for i in indexes:
            if i not in Ss:
                names.append(name + "_" + i + "_" + "".join(Ss))
    return names

def ijlS_names(name):
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for i in S:
            for l in S:
                for j in indexes:
                    if not j in S:
                        names.append(name + "_"+ i + j +l + "_" + "".join(Ss))
    return names
                    
def hS_names():
    names = []
    for S in Subsets(indexes, g):
        Ss = list(S)
        Ss.sort()
        for j in indexes:
            if not j in S:
                names.append("h_" + "".join(Ss) + "_" + j) 
    return names
        
    
Uring = PolynomialRing(QQ, two_index_names("al") + two_index_names_repeat("ga") +  two_index_names("a") + two_index_names("d") + two_index_names("t") + two_index_names("v") + two_index_names("r") + two_index_names_repeat("del") + two_index_names("b") + two_index_names("be") + two_index_names_repeat("ep") + two_index_names("psi")  + two_index_names("u") + two_index_names("g") + iS_names("pi") + two_index_names("k") + iS_names("s") 
                      + jjpS_names("c") + jjpS_names("cc") + ijS_names("a") + ijS_names("ac") + jjpS_names("hshsc") + ijS_names("b") +ijlS_names("d") + ijS_names("fhsc") + ijS_names("e") + ijS_names("r") + ijlS_names("s") +ijS_names("hhsc")
                      + two_index_names_repeat("theta") + two_index_names_repeat("pi") + inotinS_names("x"))

Rring = PolynomialRing(Uring, iS_names("f") + iS_names("h")+ hS_names())
    
Udict = Uring.gens_dict()
Rdict = Rring.gens_dict()
ExpandRing.<t> = LaurentSeriesRing(Uring)

def v(letter, a, b = "", c = "", d = None):
    #if g==2 and letter in ["c","e","ell"] and not type(c) == list:
    #    return 0
    
    #Some normalization
    if letter == "d" and type(d) == list:
        if a == d[0] and c == d[0]:
            return 0
    
    if letter == "h" and type(a) == list:
        return Rdict["h_" + "".join(a) + "_" + b]
    if type(c) == list:
        return Udict[letter + "_" + a + b + "_" + "".join(c)]
    if type(b) == list:
        try:
            return Udict[letter + "_" + a + "_" + "".join(b)]
        except:
            return Rdict[letter + "_" + a + "_" + "".join(b)]
    
    if not d is None:
        return Udict[letter + "_" + a + b + c + "_" + "".join(d)]
    try:
        return Udict[letter +"_"+ a + b +c]
    except KeyError:
        return Rdict[letter +"_"+ a + b +c]

We now create the relations.

In [4]:
def make_rel(lh, i, j, S = None):
    """
    lh is one of "ff", "fh", "hh", or "h2"
    i and j are the indexes appearing in the left hand side
    S is the set of indexes with size equal to g, must contain i,j, defaults to [i,j]

    j must be None in the case of "h2".
    """
    if S is None:
        if j is not None and g==2:
            S = [i,j]
        else:
            raise Exception("Not able to automatically find S.")

    if not j is None:
        notij = copy(S)
        notij.remove(i)
        notij.remove(j)
         
    S.sort()

    if lh == "ff":
        return ( v("f",i,S)*v("f",j,S) , v("al",j,i)*v("h",i,S) + v("al",i,j)*v("h",j,S) + v("ga",j,i)*v("f",i,S) + v("ga",i,j)*v("f",j,S) + sum((v("c",i,j,k)*v("f",k,S) for k in notij)) + v("a",i,j) )

    elif lh == "fh":
        return (v("f",i,S)*v("h",j,S) , v("d",i,j)*v("f",j,S)**2 + v("t",j,i)*v("h",i,S) + v("v",i,j)*v("h",j,S) + v("r",j,i)*v("f",i,S) + v("del",i,j)*v("f",j,S) + sum(( v("e", i,j,k)*v("f",k,S) for k in notij)) + v("b",i,j))

    elif lh == "hh":
        return (v("h",i,S)*v("h",j,S) , v("be",j,i)*v("f",i,S)**2 + v("be",i,j)*v("f",j,S)**2 + v("ep",j,i)*v("h",i,S) + v("ep",i,j)*v("h",j,S) + v("psi",j,i)*v("f",i,S) + v("psi",i,j)*v("f",j,S) + sum((v("ell",i,j,k)*v("f",k,S) for k in notij)) + v("u",i,j))

    elif lh == "h2":
        if not j is None:
            raise Exception("j must be None")

        noti = copy(S)
        noti.remove(i)

        return (v("h",i,S)**2, v("f",i,S)**3 + sum(( v("g",i,k)*v("h",k,S) for k in noti)) + v("pi",i,S)*v("f",i,S) + sum((v("k",i,k)*v("f",k,S) for k in noti)) + v("s",i,S))


First, we want to check the following.
### Lemma
The ring $\mathcal U^{ns}_{2,n}(ij)$ is generated by ten variables $\alpha_{ij}$, $\beta_{ij}$, $\gamma_{ij}$, $\epsilon_{ij}$, $\pi_i$.

We know that the ring is generated by all the coefficients appearing in the equations on page 9 of "... as moduli of $A_\infty$-structres". There are 23 letters appearing there. 3 of them ($c$, $e$, and $l$) do not occur in genus 2, and $q$, $r$, and $u$ (with one subscript), we make 0 by adding constants to $f$ and $g$. Hence, we need to eliminate 12 letters. 

By symmetry we will just check the case $i,j = 1,2$. We use the Buchberger algorithm to find relations.

In [5]:
lms = []
#the leading monomials
S = ["1","2"]
print "Warning: S is hard coded here!!!!"

for i, j in Combinations(S,2):
    lms.append( (v("f",i,S)*v("f",j,S), make_rel , ("ff", i, j,S)))
    lms.append( (v("h",i,S)*v("h",j,S), make_rel, ("hh", i, j,S)))

for i in S:
    lms.append( (v("h",i,S)^2, make_rel , ("h2", i, None, S)))  
    for j in S:
        if j==i:
            continue
        lms.append( (v("f",i,S)*v("h",j,S), make_rel,("fh", i, j,S)))

def doS(r1,r2):
    """
    Implements the Buchberger algorithm to find relations between the coefficients.
    """
    L = lcm(r1[0].change_ring(QQ), r2[0].change_ring(QQ))
    factor1 = L.quo_rem(r1[0])[0]
    factor2 = L.quo_rem(r2[0])[0]
    S = factor1 * r1[1] - factor2 * r2[1]
    S1 = S
    done = False
    while not done:
        newS = 0
        for m in S1.monomials():
            can_be_reduced = False
            for lm in lms:
                if lm[0].divides(m):
                    can_be_reduced = True
                    #newS += make_rel(lm[1], lm[2], lm[3])[1] *(m/lm[0]).numerator()*S1.monomial_coefficient(m)
                    newS += (lm[1](*lm[2]))[1] * (m/lm[0]).numerator()*S1.monomial_coefficient(m)
                    break
            if not can_be_reduced:
                newS += m*S1.monomial_coefficient(m)


        if S1 == newS:
            done = True
        else:
            #print "need to go again"
            S1 = newS

    #print "Relations Found:"
    for c in newS.coefficients():
        yield c

monos_to_save = ["al_12","al_21", "be_12", "be_21", "ga_12", "ga_21", "pi_1", "pi_2", "ep_12", "ep_21"]
def eliminate_vars(r1,r2):
    for rrr in doS(r1,r2):
        do_monos =[]
        for m in rrr.monomials():
            if m.degree() == 1:
                do_monos.append(m)

        doe_monos = []
        for m in do_monos:
            if str(m) not in monos_to_save:
                doe_monos.append(m)

        if len(doe_monos) == 0:
            if len(do_monos) >0:
                raise Exception("Must eliminate reserved var!!!!!")
            #otherwise do nothing, can't eliminate a ver
        else:

            m = doe_monos[0]

            #rels_found[m] = m-1/rrr.monomial_coefficient(m)*rrr
            print "Adding relations of the form:",m, "=",m-1/rrr.monomial_coefficient(m)*rrr
            for a,b in make_all_subs([str(m), str(m-1/rrr.monomial_coefficient(m)*rrr)]): 
                #Don't love this method of making sure we don't double eliminate a relation...
                if len(str(m)) == len(str(eval(a))):
                    Udict[str(eval(a))] = eval(b)
            
            reduceUdict()

            
def reduceUdict():
    
    vUdict = {Uring(k):val for k,val in Udict.items()}
    for k, val in Udict.items(): 
        Udict[k] = val.subs(vUdict)
        
import re
def make_all_subs(strs):
    indexes_involved = set()
    for st in strs:
        for match in re.finditer(r"[a-z]+_(\d+)(?:_(\d+))?",st):
            indexes_involved.update(match.group(1))
            if not match.group(2) is None:
                indexes_involved.update(match.group(2))

    indexes_involved = list(indexes_involved)

    for s in Subsets(indexes, len(indexes_involved)):
        for p in Permutations(s):

            def permute_indices(matchobj):
                #Tracer()()
                new_indexes = []
                for i in matchobj.group(2):
                    new_indexes += p[indexes_involved.index(i)]
                if matchobj.group(3) is None:
                    return "v('" + matchobj.group(1) + "', '" +  "','".join(new_indexes) + "')"
                #else
                new_indexes2 = []
                for i in matchobj.group(3):
                    new_indexes2 += p[indexes_involved.index(i)]
                    
                if matchobj.group(1) == "h":
                    #it is an S, j type
                    new_indexes.sort()
                    return "v('" + matchobj.group(1) + "', " + str(new_indexes) + "', '" + ", '".join(new_indexes2) + "')"
                #else it is a i, S type
                new_indexes2.sort()
                return "v('" + matchobj.group(1) + "', '" +  "','".join(new_indexes) + "', " + str(new_indexes2) + ")"
                

            yield tuple((re.sub(r"([a-z]+)_(\d+)(?:_(\d+))?", permute_indices, str).replace("^","**") for str in strs))
            #yield tuple((re.sub(r"([a-z]+)_([0-9])([0-9])?([0-9])?", permute_indices, str).replace("^","**") for str in strs))

Doing Buchberger on the $f_1f_2$ and $f_1h_2$ equations elimnates $d_{ij}$, $\delta_{12}$, 3 more.

In [6]:
eliminate_vars(make_rel("ff","1","2"),make_rel("fh","1","2"))

Adding relations of the form: d_12 = al_12
Adding relations of the form: del_12 = ga_21*d_12 + al_21*be_12
Adding relations of the form: v_12 = ga_12
Adding relations of the form: b_12 = -t_21*r_12 - ga_12*r_21 + ga_21*del_12 + al_21*psi_12 + al_12*pi_2_12
Adding relations of the form: a_12 = t_12*t_21 - ga_21*v_12 + al_12*r_21 - al_21*ep_12


Find 4 more

In [7]:
eliminate_vars(make_rel("hh","1","2"),make_rel("fh","1","2"))

Adding relations of the form: t_21 = be_21
Adding relations of the form: psi_21 = 3*al_12*al_21*ga_21 + ga_12*be_21
Adding relations of the form: r_21 = -al_12*al_21^2 + ep_21
Adding relations of the form: u_12 = 3*al_12*al_21*ga_12*ga_21 + al_12*al_21*t_12*t_21 + 2*al_12^2*al_21*r_21 + 3*al_12^2*ga_21*be_21 - al_12*al_21^2*ep_12 - ga_21^2*be_12 - r_21*ep_12 - ga_21*psi_12 + ga_12*psi_21 + t_21*pi_1_12


And here are the last 3:

In [8]:
eliminate_vars(make_rel("hh","1","2"),make_rel("h2","1",None,["1","2"]))

Adding relations of the form: g_12 = -al_12^3
Adding relations of the form: k_12 = -3*al_12^2*ga_12 + be_12^2
Adding relations of the form: s_1_12 = 2*al_12^4*al_21^2 - 6*al_12^2*ga_12*ga_21 - al_12^2*be_12*be_21 - 2*al_12^3*ep_21 - al_12*al_21^2*g_12 - ga_12^3 + 2*ga_21*be_12^2 + ep_12^2 + ep_21*g_12 - ga_12*pi_1_12


Maybe you are worried about cyclic dependencies, but check out the `Udict` to make sure all is legit:

In [9]:
for k, val in Udict.items():
    print "~~~", k, "=", val

~~~ al_12 = al_12
~~~ t_21 = be_21
~~~ pi_22_12 = pi_22_12
~~~ r_12 = -al_12^2*al_21 + ep_12
~~~ s_2_12 = 3*al_12^2*al_21^4 - 6*al_21^2*ga_12*ga_21 - al_21^2*be_12*be_21 - 3*al_21^3*ep_12 - ga_21^3 + 2*ga_12*be_21^2 + ep_21^2 - ga_21*pi_2_12
~~~ ep_12 = ep_12
~~~ g_12 = -al_12^3
~~~ pi_1_12 = pi_1_12
~~~ pi_2_12 = pi_2_12
~~~ del_21 = al_21*ga_12 + al_12*be_21
~~~ be_21 = be_21
~~~ u_12 = -2*al_12^3*al_21^3 + 3*al_12*al_21*ga_12*ga_21 + 3*al_12^2*ga_21*be_21 + al_12*al_21*be_12*be_21 + 2*al_12^2*al_21*ep_21 - 2*ga_21^2*be_12 + ga_12^2*be_21 - ep_12*ep_21 + be_21*pi_1_12
~~~ a_21 = -al_12^2*al_21^2 - ga_12*ga_21 + be_12*be_21 + al_21*ep_12 - al_12*ep_21
~~~ theta_12 = theta_12
~~~ psi_12 = 3*al_12*al_21*ga_12 + ga_21*be_12
~~~ ep_11_12 = ep_11_12
~~~ theta_11_12 = theta_11_12
~~~ v_21 = ga_21
~~~ b_21 = 4*al_12^2*al_21*ga_21 + al_12*al_21^2*be_12 + al_21*ga_12^2 + 2*al_12*ga_12*be_21 - ga_21*ep_12 - be_12*ep_21 + al_21*pi_1_12
~~~ ga_12 = ga_12
~~~ ga_22_12 = ga_22_12
~~~ k_21 = -3*al_2

In [10]:
rels = [make_rel("ff","1","2"), make_rel("fh","1","2"), make_rel("fh","2","1"), make_rel("hh","1","2"), make_rel("h2","1",None,["1","2"]), make_rel("h2","2",None,["1","2"])]
rels2 = [r[0]-r[1] for r in rels]

F = QQ
RR.<f_1_12,f_2_12,h_1_12,h_2_12> = AffineSpace(4,F)

c_point_dict ={
    v("al","1","2") :-1,
    v("al","2","1") :1,
    v("be","1","2") : 3/2,
    v("be","2","1") :3/2,
    v("ga","1","2") : 5/4,
    v("ga","2","1") :5/4,
    v("ep","1","2") :-5/4,
    v("ep","2","1") :5/4,
    v("pi","1",["1","2"]) : -3/16,
    v("pi","2",["1","2"]) : -3/16,
}

rels3 = [r.map_coefficients(lambda coef: coef.subs(c_point_dict)) for r in rels2]
#rels4 = [RR(r) for r in rels3]

X = RR.subscheme(rels3)

print "dim: ", X.dimension()
print "smooth: ", X.is_smooth()
print "hilb p: ", X.defining_ideal().homogenize().hilbert_polynomial()
print
for Xi in X.irreducible_components():
    print "  ", Xi
    print "  dim: ", Xi.dimension()
    print "  smooth: ", Xi.is_smooth()
    print "  hilb p: ", Xi.defining_ideal().homogenize().hilbert_polynomial()
    if Xi.dimension() == 1:
        C = Curve(Xi)
        print "  genus: ", C.genus()
        if C.is_singular():
            print "  sing points", C.singular_points()  

for Xi, Xj in Subsets(X.irreducible_components(),2):
    inters = Xi.intersection(Xj)
    print "  intersection, dim:",inters.dimension()
    print "  smooth: ", inters.is_smooth()
    print "  hilb p: ", inters.defining_ideal().homogenize().hilbert_polynomial()
    print "  irr:", inters.is_irreducible()
    if Xi.dimension() == 1 and Xj.dimension() == 1:
        Ci = Curve(Xi)
        Cj = Curve(Xj)
        for p in Ci.intersection_points(Cj):
            try:
                print p, "mult ", Ci.intersection_multiplicity(Cj,p)
            except:
                print "multiplicity not computed"

    print


dim:  1
smooth:  False
hilb p:  6*t - 1

   Closed subscheme of Affine Space of dimension 4 over Rational Field defined by:
  12*f_2_12 - 8*h_2_12 - 5,
  16*h_2_12^2 - 36*f_1_12 - 40*h_2_12 + 43,
  16*f_1_12*h_2_12 - 20*f_1_12 - 24*h_1_12 + 4*h_2_12 - 5,
  24*f_1_12^2 - 16*h_1_12*h_2_12 - 6*f_1_12 + 20*h_1_12 - 3
  dim:  1
  smooth:  True
  hilb p:  3*t + 1
  genus:  0
   Closed subscheme of Affine Space of dimension 4 over Rational Field defined by:
  12*f_1_12 + 8*h_1_12 - 5,
  16*h_1_12^2 - 36*f_2_12 + 40*h_1_12 + 43,
  16*f_2_12*h_1_12 + 20*f_2_12 + 4*h_1_12 - 24*h_2_12 + 5,
  24*f_2_12^2 - 16*h_1_12*h_2_12 - 6*f_2_12 - 20*h_2_12 - 3
  dim:  1
  smooth:  True
  hilb p:  3*t + 1
  genus:  0
  intersection, dim: 0
  smooth:  False
  hilb p:  3
  irr: False
(3/4, 3/4, -1/2, 1/2) mult  multiplicity not computed



In [ ]:
the_plane ={
    h_2_12 : 1/40 (16*h_2_12^2 - 36*f_1_12  + 43),
    f_2_12 : 
}

In [21]:
pit = dict()
for i,j in [("1","2"),("2","1")]:
    pit[i] = v("pi",i,["1","2"]) + 3*v("al",i,j)^2*v("ga",j,i) + 3*v("ga",i,j)^2 + 3*v("al",i,j)*v("al",j,i)*v("be",i,j)
    
M = Matrix([[v("al","1","2") , v("al","2","1"), v("be","1","2") , v("be","2","1") ],
            [2*v("ep","1","2") , 2*v("ep","2","1"), pit["1"] , pit["2"]]])
M.subs(c_point_dict)
print M
print M.rank()

[                                                      al_12                                                       al_21                                                       be_12                                                       be_21]
[                                                    2*ep_12                                                     2*ep_21 3*al_12^2*ga_21 + 3*al_12*al_21*be_12 + 3*ga_12^2 + pi_1_12 3*al_21^2*ga_12 + 3*al_12*al_21*be_21 + 3*ga_21^2 + pi_2_12]
2


In [12]:
X

Closed subscheme of Affine Space of dimension 4 over Rational Field defined by:
  f_1_12*f_2_12 - 5/4*f_1_12 - 5/4*f_2_12 - h_1_12 + h_2_12 + 5/16,
  f_2_12^2 + f_1_12*h_2_12 - 9/4*f_1_12 - 1/4*f_2_12 - 3/2*h_1_12 - 5/4*h_2_12 + 13/16,
  -f_1_12^2 + f_2_12*h_1_12 + 1/4*f_1_12 + 9/4*f_2_12 - 5/4*h_1_12 - 3/2*h_2_12 - 13/16,
  -3/2*f_1_12^2 - 3/2*f_2_12^2 + h_1_12*h_2_12 + 15/8*f_1_12 + 15/8*f_2_12 - 5/4*h_1_12 + 5/4*h_2_12 - 17/8,
  -f_1_12^3 + h_1_12^2 + 3/16*f_1_12 + 3/2*f_2_12 - h_2_12 - 19/32,
  -f_2_12^3 + h_2_12^2 + 3/2*f_1_12 + 3/16*f_2_12 + h_1_12 - 19/32

In [13]:
make_rel("ff","1","2")

(f_1_12*f_2_12,
 ga_21*f_1_12 + ga_12*f_2_12 + al_21*h_1_12 + al_12*h_2_12 - al_12^2*al_21^2 - ga_12*ga_21 + be_12*be_21 - al_21*ep_12 + al_12*ep_21)

In [14]:
rels2

[f_1_12*f_2_12 + (-ga_21)*f_1_12 + (-ga_12)*f_2_12 + (-al_21)*h_1_12 + (-al_12)*h_2_12 + al_12^2*al_21^2 + ga_12*ga_21 - be_12*be_21 + al_21*ep_12 - al_12*ep_21,
 (-al_12)*f_2_12^2 + f_1_12*h_2_12 + (al_12*al_21^2 - ep_21)*f_1_12 + (-al_12*ga_21 - al_21*be_12)*f_2_12 + (-be_21)*h_1_12 + (-ga_12)*h_2_12 - 4*al_12*al_21^2*ga_12 - al_12^2*al_21*be_21 - al_12*ga_21^2 - 2*al_21*ga_21*be_12 + be_21*ep_12 + ga_12*ep_21 - al_12*pi_2_12,
 (-al_21)*f_1_12^2 + f_2_12*h_1_12 + (-al_21*ga_12 - al_12*be_21)*f_1_12 + (al_12^2*al_21 - ep_12)*f_2_12 + (-ga_21)*h_1_12 + (-be_12)*h_2_12 - 4*al_12^2*al_21*ga_21 - al_12*al_21^2*be_12 - al_21*ga_12^2 - 2*al_12*ga_12*be_21 + ga_21*ep_12 + be_12*ep_21 - al_21*pi_1_12,
 (-be_21)*f_1_12^2 + (-be_12)*f_2_12^2 + h_1_12*h_2_12 + (-3*al_12*al_21*ga_21 - ga_12*be_21)*f_1_12 + (-3*al_12*al_21*ga_12 - ga_21*be_12)*f_2_12 + (-ep_21)*h_1_12 + (-ep_12)*h_2_12 + 2*al_12^3*al_21^3 - 3*al_12*al_21*ga_12*ga_21 - 3*al_12^2*ga_21*be_21 - al_12*al_21*be_12*be_21 - 2*al_12^2*al_

In [15]:
rels = [make_rel("ff","1","2"), make_rel("fh","1","2"), make_rel("fh","2","1"), make_rel("hh","1","2"), make_rel("h2","1",None,["1","2"]), make_rel("h2","2",None,["1","2"])]
rels2 = [(r[0].map_coefficients(lambda coef: coef.subs(c_point_dict)), r[1].map_coefficients(lambda coef: coef.subs(c_point_dict))) for r in rels]



In [16]:
rels2

[(f_1_12*f_2_12, 5/4*f_1_12 + 5/4*f_2_12 + h_1_12 - h_2_12 - 5/16),
 (f_1_12*h_2_12,
  -f_2_12^2 + 9/4*f_1_12 + 1/4*f_2_12 + 3/2*h_1_12 + 5/4*h_2_12 - 13/16),
 (f_2_12*h_1_12,
  f_1_12^2 + (-1/4)*f_1_12 + (-9/4)*f_2_12 + 5/4*h_1_12 + 3/2*h_2_12 + 13/16),
 (h_1_12*h_2_12,
  3/2*f_1_12^2 + 3/2*f_2_12^2 + (-15/8)*f_1_12 + (-15/8)*f_2_12 + 5/4*h_1_12 + (-5/4)*h_2_12 + 17/8),
 (h_1_12^2, f_1_12^3 + (-3/16)*f_1_12 + (-3/2)*f_2_12 + h_2_12 + 19/32),
 (h_2_12^2, f_2_12^3 + (-3/2)*f_1_12 + (-3/16)*f_2_12 - h_1_12 + 19/32)]

In [20]:
for c1,c2 in Combinations(rels2, 2):
    print "doing", c1,c2
    for r in doS(c1,c2):
        if r.subs(c_point_dict) != 0:
            print r
            print r.subs(c_point_dict)
            print
    print
        

doing (f_1_12*f_2_12, 5/4*f_1_12 + 5/4*f_2_12 + h_1_12 - h_2_12 - 5/16) (f_1_12*h_2_12, -f_2_12^2 + 9/4*f_1_12 + 1/4*f_2_12 + 3/2*h_1_12 + 5/4*h_2_12 - 13/16)

doing (f_1_12*f_2_12, 5/4*f_1_12 + 5/4*f_2_12 + h_1_12 - h_2_12 - 5/16) (f_2_12*h_1_12, f_1_12^2 + (-1/4)*f_1_12 + (-9/4)*f_2_12 + 5/4*h_1_12 + 3/2*h_2_12 + 13/16)

doing (f_1_12*f_2_12, 5/4*f_1_12 + 5/4*f_2_12 + h_1_12 - h_2_12 - 5/16) (h_1_12*h_2_12, 3/2*f_1_12^2 + 3/2*f_2_12^2 + (-15/8)*f_1_12 + (-15/8)*f_2_12 + 5/4*h_1_12 + (-5/4)*h_2_12 + 17/8)

doing (f_1_12*f_2_12, 5/4*f_1_12 + 5/4*f_2_12 + h_1_12 - h_2_12 - 5/16) (h_1_12^2, f_1_12^3 + (-3/16)*f_1_12 + (-3/2)*f_2_12 + h_2_12 + 19/32)

doing (f_1_12*f_2_12, 5/4*f_1_12 + 5/4*f_2_12 + h_1_12 - h_2_12 - 5/16) (h_2_12^2, f_2_12^3 + (-3/2)*f_1_12 + (-3/16)*f_2_12 - h_1_12 + 19/32)

doing (f_1_12*h_2_12, -f_2_12^2 + 9/4*f_1_12 + 1/4*f_2_12 + 3/2*h_1_12 + 5/4*h_2_12 - 13/16) (f_2_12*h_1_12, f_1_12^2 + (-1/4)*f_1_12 + (-9/4)*f_2_12 + 5/4*h_1_12 + 3/2*h_2_12 + 13/16)

doing (f_1_12

In [18]:
Combinations?
